In [1]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

# 1. 데이터 가져오기


In [2]:
import pandas as pd
df = pd.read_csv("C:/ai_x/download/ShareData/부동산_250213/최종전국평당분양가격(결측치제외).csv",encoding='cp949')
df.head()

,지역명,연도,월,평당분양가격
0,서울,2013,12,18189.0
1,부산,2013,12,8111.0
2,대구,2013,12,8080.0
3,인천,2013,12,10204.0
4,광주,2013,12,6098.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역명     2176 non-null   object 
 1   연도      2176 non-null   int64  
 2   월       2176 non-null   int64  
 3   평당분양가격  2176 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 68.1+ KB


In [51]:
df

,지역명,연도,월,평당분양가격,지역명2n,연도n,월n,연도s,월s
0,서울,2013,12,18189.0,8,0.0,1.000000,-1.875367,1.62196
1,부산,2013,12,8111.0,7,0.0,1.000000,-1.875367,1.62196
2,대구,2013,12,8080.0,5,0.0,1.000000,-1.875367,1.62196
3,인천,2013,12,10204.0,11,0.0,1.000000,-1.875367,1.62196
4,광주,2013,12,6098.0,4,0.0,1.000000,-1.875367,1.62196
...,...,...,...,...,...,...,...,...,...
2171,전북,2024,8,12058.2,13,1.0,0.636364,1.664199,0.46374
2172,전남,2024,8,13120.8,12,1.0,0.636364,1.664199,0.46374
2173,경북,2024,8,13827.0,3,1.0,0.636364,1.664199,0.46374
2174,경남,2024,8,13252.8,2,1.0,0.636364,1.664199,0.46374


# 2. 지역명의 라벨 인코딩
- 지역명을 라벨인코딩한 지역명 2 필드 추가 
- 분석할 경우 원핫인코딩까지 할 것을 추천


In [ ]:
'''
1. 지역명만 numpy로 추출
2. 추출된 지역명 라벨인코딩
3. 라벨인코딩 데이터프레임으로 변경
4. df에 지역명2n으로 저장하고 재할당
'''

In [4]:
from tensorflow.keras.utils import to_categorical # 분류분석시 원핫인코딩 (전처리 작업) 
from tensorflow.keras.models import Sequential    # 모델 생성
from tensorflow.keras.layers import Dense,Input 
from sklearn.preprocessing import LabelEncoder    # 라벨인코딩
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd

# 지역명만 numpy로 추출
new_city = df['지역명'].values
print('원 데이터 :', new_city)

# 추출된 지역명 라벨인코딩 작업
le = LabelEncoder()  # 라벨인코딩 객체 생성
le.fit(new_city)
labeled_data = le.fit_transform(new_city)
print('라벨인코딩된 데이터 :\n', labeled_data)
# one_hot_encoded_data = to_categorical(labeled_data)
# print('원핫인코딩된 데이터:\n', one_hot_encoded_data)

#라벨인코딩 데이터(labeled_data)를 데이터 프레임으로 변경
new_df = pd.DataFrame(labeled_data)
new_df

#df에 지역명2n으로 저장하기
df['지역명2n']=new_df

원 데이터 : ['서울' '부산' '대구' ... '경북' '경남' '제주']
라벨인코딩된 데이터 :
 [ 8  7  5 ...  3  2 14]


# 3. normalization 스케일 조정
- 입력변수와 타겟변수 따로 스케일 조정(MinMaxScaler이용)
- 지역명 2n, 연도n, 월n 필드 추가 

## 입력변수('지역명2n','연도','월') 스케일 조정

In [37]:
from sklearn.preprocessing import MinMaxScaler 

x_data = df[['연도','월','지역명2n']].values
x_data
y_data = df[['평당분양가격']].values
y_data

#x_data 스케일 조정
scaler_x = MinMaxScaler() # x_data를 정규화 시킬 객체 
scaled_x_data = scaler_x.fit_transform(x_data)
scaled_x_data
#x_data 데이터 프레임으로 변환 후 연도n, 월n 필드 추가 
new_x = pd.DataFrame(scaled_x_data)
new_x 
df['연도n'] = new_x.iloc[:,:1]
df['월n'] = new_x.iloc[:,1:2]

array([[0.        , 1.        , 0.5       ],
       [0.        , 1.        , 0.4375    ],
       [0.        , 1.        , 0.3125    ],
       ...,
       [1.        , 0.63636364, 0.1875    ],
       [1.        , 0.63636364, 0.125     ],
       [1.        , 0.63636364, 0.875     ]])

# 4. standardization 스케일 조정
- 입력변수와 타겟변수 따로 스케일 조정(StandardScaler이용)
- 지역명 2s, 연도s, 월s필드 추가 

In [50]:
from sklearn.preprocessing import StandardScaler

x_data = df[['연도','월','지역명2n']].values
x_data
y_data = df[['평당분양가격']].values
y_data

scaler_x = StandardScaler() #x_data를 표준화할 객체
scaler_y = StandardScaler() #y_data를 표준화할 객체 
scaled_x_data = scaler_x.fit_transform(x_data)
# scaled_y_data = scaler_y.fit_transform(y_data)
# scaled_y_data
news_x = pd.DataFrame(scaled_x_data)
df['연도s'] = news_x.iloc[:,:1]
df

df['월s'] = news_x.iloc[:,1:2]


In [54]:
df.to_csv('data/test_scale.csv', index=False, encoding='utf-8')